In [1]:
import numpy as np
celeba = np.load('drive/My Drive/celeba.npy')


In [ ]:
#load attributes
attribute = np.loadtxt('drive/My Drive/list_attr_celeba.txt',skiprows=2,unpack=True,usecols=range(1,41))
attribute = np.transpose(attribute)
attribute[attribute==-1]=0
print(attribute.shape)

In [3]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Input, Activation, Lambda
from tensorflow.keras.layers import Reshape, Multiply
from tensorflow.keras.layers import Flatten, concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.nn import sigmoid_cross_entropy_with_logits
from tensorflow.math import multiply
from tensorflow.keras import backend
from matplotlib import pyplot

In [4]:

def expand_label(x):
  x = backend.expand_dims(x,axis=1)
  x = backend.expand_dims(x,axis=1)
  x = backend.tile(x,[1,32,32,1])
  return x

# define the standalone discriminator model
def define_discriminator(in_shape=(64,64,3),n_labels = 40):
  
  init = RandomNormal(stddev = 0.02)
  
  #label input
  
  in_label = Input(shape=(n_labels,))
  
  #image input
  in_image = Input(shape=in_shape)

  
  label_input = Lambda(expand_label)(in_label)
  
  
  
  # downsample to 32x32
  model = Conv2D(64, (5,5), padding='same',strides=(2,2),kernel_initializer=init,input_shape=in_shape)(in_image)
  model = LeakyReLU(alpha=0.2)(model)
  

  label_input = Lambda(expand_label)(in_label)

  #concatenate label input with the output of the first hidden layer
  x = concatenate([model, label_input], axis=3)
  
  
  model = Conv2D(128, (5,5),kernel_initializer=init, padding='same',input_shape=(32,32,104))(x)
  model = LeakyReLU(alpha=0.2)(model)
  

  # downsample to 16x16
  model = Conv2D(128, (5,5), strides=(2,2),kernel_initializer=init,padding='same')(model)
  
  model = LeakyReLU(alpha=0.2)(model)
  
  # downsample to 8x8
  model = Conv2D(256, (5,5), strides=(2,2), kernel_initializer=init,padding='same')(model)
  
  model = LeakyReLU(alpha=0.2)(model)
  

  # downsample to 4x4
  model = Conv2D(512, (5,5), strides=(2,2), padding='same')(model)
  
  model = LeakyReLU(alpha=0.2)(model)
  model = Dropout(0.4)(model)
  
  # classifier
  model = Flatten()(model)
  

  out1 = Dense(1, activation='sigmoid')(model)
  
  # compile model
  model_fin = Model([in_image,in_label], out1)
  opt = Adam(lr=0.0002, beta_1=0.5)
  
  model_fin.compile(loss='binary_crossentropy', optimizer=opt)
  
  return model_fin,opt

  

In [5]:
# define the standalone generator model
def define_generator(latent_dim,n_labels = 40):
    init = RandomNormal(stddev=0.02)
    in_label = Input(shape=(n_labels,))
    
    

    
    in_latent = Input(shape=(latent_dim,))
    
    #concatenate latent vector and label
    x = concatenate([in_latent,in_label])
    

    # foundation for 4X4 feature maps

    n_nodes = 512 * 4 * 4
    dim_inp = latent_dim + n_labels 
    model = Dense(n_nodes, input_dim=dim_inp,kernel_initializer= init)(x)
    
    model = LeakyReLU(alpha=0.2)(model)
   
    model = Reshape((4, 4, 512))(model)
    # upsample to 8X8
   
    model = Conv2DTranspose(256, (4,4), strides=(2,2), padding='same',kernel_initializer= init)(model)
    
    model = LeakyReLU(alpha=0.2)(model)
    
    # upsample to 16x16
    model = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer= init)(model)
    
    model = LeakyReLU(alpha=0.2)(model)
    
    # upsample to 32x32
    model = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer= init)(model)
    
    model = LeakyReLU(alpha=0.2)(model)
    
    # upsample to 64x64
    model = Conv2DTranspose(64, (4,4),strides=(2,2),padding='same',kernel_initializer= init)(model)
    model = LeakyReLU(alpha=0.2)(model)

    
    
    # output layer 64x64x3
    model = Conv2D(3, (5,5),padding='same',kernel_initializer= init)(model)
    
    out = Activation('tanh')(model)

    model_fin = Model([in_latent, in_label], out)
    
    
    
    return model_fin

In [6]:
def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  # connect them
  gen_noise, gen_label = g_model.input
  # add generator
  gen_output = g_model.output
  # add the discriminator
  gan_output = d_model([gen_output,gen_label])
  # compile model
  
  model = Model([gen_noise,gen_label], gan_output)
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy',
  optimizer=opt)  
  
  return model

In [7]:
# select real samples
import copy
def generate_real_samples(dataset,label, n_samples):
  # choose random instances
  ix = randint(0, dataset.shape[0], n_samples)
  # retrieve selected images with resepctive labels
  X, labels = dataset[ix], label[ix]
  
  X = (X.astype('float32')-127.5)/127.5
  
  y = ones((n_samples, 1))
  return [X,labels], y

In [8]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim,labels_dataset, n_samples, n_labels = 40):
  
  # generate points in the latent space
  x_input = randn(latent_dim * n_samples)
  
  # reshape into a batch of inputs for the network
  x_input = x_input.reshape(n_samples, latent_dim)

  #select random labels from the dataset
  idx = randint(0, labels_dataset.shape[0], n_samples)
  labels = labels_dataset[idx]
  
  
  return [x_input,labels]

In [9]:
# use the generator to generate n fake examples, with labels
def generate_fake_samples(g_model, latent_dim,labels_dataset, n_samples,n_labels = 40):
  
  # generate points in latent space
  x_input,labels = generate_latent_points(latent_dim,labels_dataset, n_samples,n_labels)
  
  # predict outputs
  X = g_model.predict([x_input,labels])
  
  # create fake class labels 
  y = zeros((n_samples, 1)) 
  return [X,labels], y

In [10]:
# create and save a plot of generated images
def save_plot(examples, epoch,labels_fake, n=4):
  # scale from [-1,1] to [0,1]
  examples = (examples + 1) / 2.0
  # plot images
  for i in range(n * n):
  # define subplot
    pyplot.subplot(n, n, 1 + i)
  # turn off axis
    pyplot.axis('off')
  # plot raw pixel data
    pyplot.imshow(examples[i])
  # save plot to file
  labels_fake = np.array(labels_fake)
  filename = 'drive/My Drive/model/fakelabelsACgan_e%03d.csv' % (epoch+1)
  np.savetxt(filename, labels_fake, delimiter=",")
  filename = 'drive/My Drive/model/generated_plotACgan_e%03d.png' % (epoch+1)
  pyplot.savefig(filename)
  pyplot.close()
  # evaluate the discriminator, plot generated images, save generator model

#generate and save a plot of fake images, save models
def save_training(epoch, g_model,d_model,gan_model, dataset,labels_dataset, latent_dim,losses, n_samples=16):
  # prepare real samples
  [x_fake,labels_fake], y_real = generate_fake_samples(g_model,latent_dim,labels_dataset, n_samples)
  # save plot

  save_plot(x_fake, epoch,labels_fake)
  # save the generator model tile file
  filename = 'drive/My Drive/model/ACgenerator_model_%03d.h5' % (epoch + 1)
  g_model.save(filename)
  filename = 'drive/My Drive/model/ACgan_model_%03d.h5' % (1)
  gan_model.save(filename)
  filename = 'drive/My Drive/model/ACd_model_%03d.h5' % (1)
  d_model.trainable = True
  for layer in d_model.layers:
    layer.trainable = True
  d_model.save(filename)
  d_model.trainable = False
  for layer in d_model.layers:
    layer.trainable = False
  
  np.savez('drive/My Drive/Losses/losses_modelfinalpre2.npz', *losses)
  



In [11]:
#training
def train(g_model, d_model, gan_model, dataset,labels_dataset,latent_dim, n_epochs=40, n_batch=64):
  half_batch = int(n_batch / 2)
  bat_per_epo = int(dataset.shape[0] / n_batch)
  
  dloss_real = []
  dloss_fake = []
  gloss = []
  
  
  
  
  # manually enumerate epochs
  for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      # get randomly selected ✬real✬ samples
      
      [X_real,labels_real], y_real = generate_real_samples(dataset,labels_dataset, half_batch)
      
      # update discriminator model weights
      d_loss1 = d_model.train_on_batch([X_real,labels_real],y_real)
      
      # generate fake examples
      [X_fake,labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, labels_dataset, half_batch)
      
      # update discriminator model weights
      d_loss2 = d_model.train_on_batch([X_fake,labels_fake],y_fake )
      
      # prepare points in latent space as input for the generator
      [X_gan,label_gan] = generate_latent_points(latent_dim,labels_dataset, n_batch)
      
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      
      # update the generator via the discriminator's error
      g_loss = gan_model.train_on_batch([X_gan,label_gan], [y_gan])
      
      #store losses
      dloss_real.append(d_loss1)
      dloss_fake.append(d_loss2)
      gloss.append(g_loss)
      

      print('>%d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f' %
      (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
      
    
      
    losses = [np.array(dloss_real),np.array(dloss_fake),np.array(gloss)]
    
    save_training(i,g_model,d_model,gan_model, dataset, labels_dataset, latent_dim,losses)
    
  

In [ ]:

# size of the latent space
latent_dim = 100
# create the discriminator
d_model,optd = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)


# train model
train(g_model, d_model, gan_model, celeba[:192599], attribute[:192599], latent_dim)

In [ ]:
from tensorflow.keras.models import load_model

#function used to load models and restart training
def load_models():
    
    
    filename = 'drive/My Drive/model/ACd_model_001.h5'
    d_model = load_model(filename, compile=True)
    d_model.trainable = True
    for layer in d_model.layers:
        layer.trainable = True
    d_model.summary()
    filename = 'drive/My Drive/model/ACgenerator_model_035.h5'
    g_model = load_model(filename, compile=True)
    g_model.summary()
    gan_model = define_gan(g_model, d_model)
    gan_model.summary()
    return d_model, g_model, gan_model